In [1]:
import shutil
import os
import pandas as pd
import glob
import random

In [8]:
samples = pd.read_csv("/Users/haleyjohnson/Desktop/biased_online_ads/files/sampled_users.csv")

In [9]:
big_path = '/Volumes/Sensitive Group Browsing/Sensitive Group Browsing/Screenshots/*.zip'
files = glob.glob(big_path)

In [11]:
targets = {}
for f in files: 
    zip_file = f.split("/")[-1]
    user = zip_file[:10]
    if user[6] == '0':
        user = user[:5] + user[6:10]
    if user[7] == '0':
        user = user[:5] + user[7:10]
    if user in samples.user.values:
        targets[user] = zip_file

In [12]:
l = list(targets.items())

In [13]:
extract_dir = '/Users/haleyjohnson/Desktop/samples'
end_path = '/Users/haleyjohnson/Desktop/sample_screenshots'
samples_path = '/Volumes/Sensitive Group Browsing/Sensitive Group Browsing/Screenshots'

failed = []

for target, file in l:
    rows = samples[samples.user == target]
    full_path = f'{samples_path}/{file}'
    try:
        shutil.unpack_archive(full_path, extract_dir = extract_dir)
    except: 
        failed.append((target, "opening"))
        continue
    # get all examples from particular folder 
    for idx, vals in rows.iterrows():
        site = str(vals['suffix'])
        files = glob.glob(f'{extract_dir}/users/{target}/screenshot_0/data/screenshots/parts/*-_{site}-part*.png')
        try:
            file = random.choice(files)
        except:
            failed.append((target, 'site_match'))
            continue
        new_name = f'{target}_{site}.png'
        os.rename(file, f"{end_path}/{new_name}")
    # delete extra data when migration is done
    shutil.rmtree(f"{extract_dir}/users/{target}")


In [17]:
failed_df = pd.DataFrame(failed, columns = ['user', 'cause'])

In [20]:
failed_df.to_csv("../../files/failed_to_sample.csv")